# 핵심이슈 추출의 성능평가

## 1. 정성적 평가 
**숫자나 수치로 나타내기 어려운 부분을, 사람의 주관이나 판단을 바탕으로 평가**
- 월별 기사 샘플 ( 약 3~5건 임의로 추출 ) 요약본을 직접 읽고 TF-IDF가 정말 핵심이슈인지 확인
- 사람이 보기에 어색하거나 의미 없는 단어가 상위에 있다면 불용어 리스트 재정비
- 주관적인 평가가 될 수 있지만 팀원들이 적당히 괜찮다 생각하면 정성적 평가로 들어갈 수 있음

In [8]:
# 개별 기사 핵심이슈 추출 검증 코드
import json
import os
import re
import random
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
from datetime import datetime
from konlpy.tag import Okt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# =========================
# 설정
# =========================
# 파일 경로 설정 (실제 경로로 수정 필요)
file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/final_preprocessing.json'
output_csv_path = './keyword_extraction_validation_results.csv'

# 샘플 개수 설정 (전체 데이터가 많을 경우 일부만 샘플링)
SAMPLE_SIZE = 10  # None으로 설정하면 전체 데이터 처리

# =========================
# 형태소 분석기
# =========================
okt = Okt()

# =========================
# 기존 함수들 재사용
# =========================
def normalize_text(t: str) -> str:
    """텍스트 정규화"""
    if not t:
        return ""
    t = t.replace("탄도 미사일", "탄도미사일")
    t = t.replace("순항 미사일", "순항미사일")
    t = t.replace("극초 음속", "극초음속")
    t = t.replace("초대형 방사포", "초대형방사포")
    return t

def pos_tokens(text: str):
    """형태소 분석"""
    text = normalize_text(text or "")
    return okt.pos(text, norm=True, stem=True)

def doc_text(a) -> str:
    """문서 텍스트 추출"""
    return normalize_text(f"{a.get('title','')} {a.get('summary','')}")

# 불용어 정의
BASE_STOP = set([
    '가','간','같은','같이','것','게다가','결국','곧','관하여','관련','관한','그','그것','그녀','그들',
    '그리고','그때','그래','그래서','그러나','그러므로','그러한','그런','그렇게','그외','근거로','기타',
    '까지도','까지','나','남들','너','누구','다','다가','다른','다만','다소','다수','다시','다음','단','단지',
    '당신','대','대해서','더군다나','더구나','더라도','더욱이','도','도로','또','또는','또한','때','때문',
    '라도','라면','라는','로','로부터','로써','를','마저','마치','만약','만일','만큼','모두','무엇','무슨',
    '무척','물론','및','밖에','바로','보다','뿐이다','사람','사실은','상대적으로','생각','설령','소위','수',
    '수준','쉽게','시대','시작하여','실로','실제','아니','아무','아무도','아무리','아마도','아울러','아직',
    '앞에서','앞으로','어느','어떤','어떻게','어디','언제','얼마나','여기','여부','역시','예','오히려',
    '와','왜','외에도','요','우리','우선','원래','위해서','으로','으로부터','으로써','을','의','의거하여',
    '의지하여','의해','의해서','의하여','이','이것','이곳','이때','이라고','이러한','이런','이렇게','이제',
    '이지만','이후','이상','이다','이전','인','일','일단','일반적으로','임시로','입장에서','자','자기','자신',
    '잠시','저','저것','저기','저쪽','저희','전부','전혀','점에서','정도','제','조금','좀','주로','주제','즉',
    '즉시','지금','진짜로','차라리','참','참으로','첫번째로','최고','최대','최소','최신','최초','통하여',
    '통해서','평가','포함한','포함하여','하지만','하면서','하여','한','한때','한번','할','할것이다','할수있다',
    '함께','해도', '돼다', '서다', '대해', '나오다', '통해', '맞다', 
])

NEWS_STOP = {"기자","연합뉴스","사진","속보","종합","자료","영상","단독","전문","인터뷰","브리핑"}

ENTITY_NOISE = {
    "북한","한국","대한민국","남한","미국","중국","일본","러시아","우크라이나","유엔","나토","NATO","EU","유럽연합",
    "푸틴","블라디미르 푸틴","바이든","조 바이든","시진핑","김정은","김여정","문재인","윤석열","쇼이구","젤렌스키","통신","중앙","보도"
}

def tokenizer_for_vectorizer(s: str):
    """TF-IDF용 토크나이저"""
    toks = []
    for w, t in okt.pos(s, norm=True, stem=True):
        if t not in ("Noun", "Verb"):
            continue
        if len(w) <= 1:
            continue
        if w in BASE_STOP or w in NEWS_STOP:
            continue
        if w.isdigit():
            continue
        toks.append(w)
    return toks

def learn_action_lexicons_single(article):
    """단일 기사에서 행동 동사와 행위 명사 추출"""
    title = article.get('title','') or ''
    summary = article.get('summary','') or ''
    p = pos_tokens(f"{title} {summary}")

    verb_set = set()
    action_nouns = set()
    drop_verbs = {"하다","되다","이다","있다"}

    for i, (w, t) in enumerate(p):
        if t == "Verb" and w not in drop_verbs and len(w) > 1:
            verb_set.add(w)
        if t == "Noun" and len(w) > 1 and w not in BASE_STOP:
            ahead = [p[j][0] for j in range(i+1, min(i+3, len(p)))]
            if "하다" in ahead or "되다" in ahead:
                action_nouns.add(w)

    return verb_set, action_nouns

def nominalize_verb(v: str) -> str:
    """동사 명사화"""
    if v.endswith("하다"):
        return v[:-2]
    if v.endswith("되다"):
        return v[:-2]
    return v

def extract_single_article_keywords(article, top_k=5):
    """단일 기사에서 핵심 키워드 추출"""
    title = article.get('title','') or ''
    summary = article.get('summary','') or ''
    
    if not title and not summary:
        return []

    # 행동 동사와 행위 명사 학습
    verb_set, action_nouns = learn_action_lexicons_single(article)
    
    # 형태소 분석
    p = pos_tokens(f"{title} {summary}")
    
    # 키워드 후보 추출
    phrase_candidates = set()
    prev_nouns = []
    
    for i, (w, t) in enumerate(p):
        if t == "Noun":
            if w not in BASE_STOP and len(w) > 1:
                prev_nouns.append(w)
                if len(prev_nouns) > 3:
                    prev_nouns = prev_nouns[-3:]

        # 동사 기반 구문 생성
        if t == "Verb" and w in verb_set:
            vnom = nominalize_verb(w)
            if vnom and len(vnom) > 1:
                # 동사만
                phrase_candidates.add(vnom)
                # 명사 + 동사
                if prev_nouns:
                    phrase_candidates.add(f"{prev_nouns[-1]} {vnom}")
                    if len(prev_nouns) >= 2:
                        phrase_candidates.add(f"{prev_nouns[-2]} {prev_nouns[-1]} {vnom}")

        # 행위 명사 기반 구문 생성
        if t == "Noun" and w in action_nouns:
            # 명사만
            phrase_candidates.add(w)
            # 앞 명사 + 행위 명사
            if prev_nouns and prev_nouns[-1] != w:
                phrase_candidates.add(f"{prev_nouns[-1]} {w}")
                if len(prev_nouns) >= 2:
                    phrase_candidates.add(f"{prev_nouns[-2]} {prev_nouns[-1]} {w}")

    # 단일 문서 TF-IDF 계산
    doc_content = doc_text(article)
    if not doc_content.strip():
        return []

    try:
        vectorizer = TfidfVectorizer(
            tokenizer=tokenizer_for_vectorizer,
            ngram_range=(1, 3),
            lowercase=False
        )
        
        # 단일 문서라서 TF만 계산 (IDF는 의미없음)
        X = vectorizer.fit_transform([doc_content])
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = X.toarray()[0]
        
        # TF 점수 딕셔너리 생성
        tf_dict = {feature_names[i]: tfidf_scores[i] for i in range(len(feature_names)) if tfidf_scores[i] > 0}
        
    except:
        tf_dict = {}

    # 엔터티 노이즈 필터링 함수
    def is_entity_only(ph: str) -> bool:
        toks = ph.split()
        if len(toks) <= 2 and any(ent in ph for ent in ENTITY_NOISE):
            return True
        ent_hits = sum(1 for t in toks if any(ent in t for ent in ENTITY_NOISE))
        return (ent_hits >= max(1, len(toks) - 1))

    # 키워드 점수 계산
    scored_phrases = []
    for phrase in phrase_candidates:
        if is_entity_only(phrase):
            continue
        if len(phrase.strip()) <= 2:
            continue
            
        # TF 점수 가져오기
        tf_score = tf_dict.get(phrase, 0.0)
        
        # 구문 길이 보너스
        length_bonus = len(phrase.split()) * 0.1
        
        # 최종 점수
        final_score = tf_score + length_bonus
        
        if final_score > 0:
            scored_phrases.append((phrase, final_score))

    # 점수순 정렬 후 상위 k개 반환
    scored_phrases.sort(key=lambda x: x[1], reverse=True)
    return [phrase for phrase, score in scored_phrases[:top_k]]

def load_articles(file_path):
    """기사 데이터 로드"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        if not isinstance(data, list):
            raise ValueError("JSON 루트는 list 여야 합니다.")
        return data
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return None
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")
        return None

def process_articles_for_validation(articles, sample_size=None):
    """검증을 위한 기사 처리"""
    if sample_size and len(articles) > sample_size:
        print(f"전체 {len(articles)}개 기사 중 {sample_size}개 랜덤 샘플링")
        # 랜덤 시드 설정 (재현 가능한 결과를 위해)
        random.seed(42)
        articles = random.sample(articles, sample_size)
    
    results = []
    
    for i, article in enumerate(tqdm(articles, desc="기사별 키워드 추출 중")):
        title = article.get('title', '') or ''
        summary = article.get('summary', '') or ''
        
        # 제목과 요약이 모두 비어있으면 스킵
        if not title.strip() and not summary.strip():
            continue
            
        # 키워드 추출
        keywords = extract_single_article_keywords(article, top_k=5)
        keywords_str = ' | '.join(keywords) if keywords else '추출된 키워드 없음'
        
        results.append({
            'article_id': i,
            'title': title.strip(),
            'summary': summary.strip(),
            'extracted_keywords': keywords_str,
            'num_keywords': len(keywords)
        })
    
    return results

# =========================
# 실행부
# =========================
def main():
    print("🔍 키워드 추출 검증 프로세스를 시작합니다.")
    
    # 1. 데이터 로드
    print("📂 데이터 로딩 중...")
    articles = load_articles(file_path)
    if not articles:
        print("❌ 데이터를 로드할 수 없습니다.")
        return
    
    print(f"✅ 총 {len(articles)}개의 기사를 로드했습니다.")
    
    # 2. 검증 처리
    print("🔄 기사별 키워드 추출을 진행합니다...")
    results = process_articles_for_validation(articles, SAMPLE_SIZE)
    
    if not results:
        print("❌ 처리할 수 있는 기사가 없습니다.")
        return
    
    # 3. DataFrame으로 변환
    df = pd.DataFrame(results)
    
    # 4. CSV로 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"💾 결과를 '{output_csv_path}'에 저장했습니다.")
    
    # 5. 간단한 통계 출력
    print("\n📊 추출 결과 통계:")
    print(f"- 총 처리된 기사 수: {len(results)}")
    print(f"- 키워드가 추출된 기사 수: {len(df[df['num_keywords'] > 0])}")
    print(f"- 키워드가 추출되지 않은 기사 수: {len(df[df['num_keywords'] == 0])}")
    print(f"- 평균 키워드 개수: {df['num_keywords'].mean():.2f}")
    
    # 6. 모든 결과 상세 출력 (5개만 처리하므로)
    print(f"\n📋 전체 추출 결과:")
    for i, row in df.iterrows():
        print(f"\n{'='*80}")
        print(f"📰 기사 {i+1}")
        print(f"제목: {row['title']}")
        print(f"요약: {row['summary'][:200]}{'...' if len(row['summary']) > 200 else ''}")
        print(f"🔑 추출된 키워드: {row['extracted_keywords']}")
        print(f"키워드 개수: {row['num_keywords']}개")
    
    return df

# 실행
if __name__ == "__main__":
    df_results = main()

🔍 키워드 추출 검증 프로세스를 시작합니다.
📂 데이터 로딩 중...
✅ 총 80810개의 기사를 로드했습니다.
🔄 기사별 키워드 추출을 진행합니다...
전체 80810개 기사 중 10개 랜덤 샘플링


기사별 키워드 추출 중: 100%|██████████| 10/10 [00:01<00:00,  8.15it/s]

💾 결과를 './keyword_extraction_validation_results.csv'에 저장했습니다.

📊 추출 결과 통계:
- 총 처리된 기사 수: 10
- 키워드가 추출된 기사 수: 10
- 키워드가 추출되지 않은 기사 수: 0
- 평균 키워드 개수: 4.80

📋 전체 추출 결과:

📰 기사 1
제목: 
요약: 또 다른 분석업체 TRM 랩스는 북한이 지난해 해킹으로 강탈한 수천만달러 규모의 가상화폐 가치가 최근 몇 주 사이 80∼85% 폭락해 현재 1천만달러(약 130억원)도 안 되는 것으로 추정했다. 지난달 시작된 가상화폐 가치의 갑작스러운 급락으로 북한이 해킹 등으로 현금을 마련하는 능력이 훼손되고 무기 개발 프로그램에 자금을 조달하는 계획도 영향을 받았을 것...
🔑 추출된 키워드: 계획 영향 받다 | 해킹 자원 쏟다 | 지적 재무부 따르다 | 자원 쏟다 | 재무부 따르다
키워드 개수: 5개

📰 기사 2
제목: 
요약: 아베 신조 일본 총리는 비핵화에 대한 분명한 언급이 선행돼야 한다는 기존의 입장을 재확인하면서 북미 정상회담에 앞서 4월 미일 정상간 대화를 제안했습니다. 일본 정부는 북미 정상회담 발표에에 일단 환영의 뜻을 표하면서도 경계심를 드러냈습니다. 그는 "비핵화 논의를 시작할 수 있는 북한의 변화를 높이 평가한다"면서도, 급변하는 한반도 정세에서 일본이 배제되는...
🔑 추출된 키워드: 대한 우려 시키다 | 분석 비핵화 앞세우다 | 입장 확인 겁니다 | 진전 경계 드러내다 | 일본 총리 이르다
키워드 개수: 5개

📰 기사 3
제목: 
요약: 北 "초대형방사포 시험발사"…추가발사 예고 북한이 어제(10일) 쏜 발사체가 '초대형 방사포'였다고 밝혔습니다. 북한은 발사체 발사 다음 날 '초대형 방사포'를 시험했다고 밝혔습니다. 지난달 24일에 이어, 이번엔 내륙을 관통하는 식으로 또다시 쏘아 올린 것입니다. <조선중앙TV> "초대형방사포시험사격은 시험사격 목적에 완전부합되었으며 무기체계완성의 다음 ...
🔑 추출된 키워드: 완전 부합

# 다른 모델과 비교하여 보기

### 어떻게 해석하면 좋을지 (발표용 포인트)

- **서로 다른 알고리즘의 관점 차이**
    - **TF-IDF**: “해당 월 문서에서 상대적으로 자주 등장하는 n-그램”을 찾아요(코퍼스 기반 빈도 가중).
    - **YAKE**: 단일 문서(여기서는 월 코퍼스 합본) 내부 분포 특성(위치/대문자/길이/출현 분산 등)을 이용한 **언어 독립적 키프레이즈**.
    - **KeyBERT**: 문서 임베딩과 후보구 임베딩의 코사인 유사도로 **의미적 근접성**을 보는 방식.
- **검증/비교 지표**
    - **자카드(구/토큰 단위)**: 결과 집합의 겹침 정도(= 일관성).
    - **스피어만 상관**: 공통 후보들의 **순위 일관성**.
    - **교집합 Top 목록**: 프락티컬하게 “모두가 중요하다고 보는 표현”을 빠르게 제시.
- **권장 해석 시나리오**
    1. **교집합 상위 키워드**는 “핵심 이슈의 신뢰 코어”로 두고,
    2. **방법별 고유 상위 키워드**는 보완적 관점(의미 중심/빈도 중심/그래프 중심)으로 **신뢰도 보강/리드 신호 탐색**에 활용하세요.
    3. 월별로 **겹침도가 낮아지는 구간**은 이슈 구성이 변하는 **전환점**으로 볼 수 있습니다.

In [2]:
!pip install krwordrank yake keybert sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [keybert]m5/7 [sentence-transformers]


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
월별 키워드 추출 성능 비교 파이프라인
- Methods: TF-IDF, TextRank(krwordrank), YAKE, KeyBERT
- Inputs:
    1) 전체 기사 JSON (list[dict])  — file_path (제목=metadata.title, 요약=summary)
    2) (가능하면) 전체코퍼스 벡터라이저 pkl — TFIDF_VECTORIZER_PATH
- Outputs (기본):
    /home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/benchmarks/
      ├─ YYYY_MM_methods_keywords.csv                (방법별 TopK 키워드)
      ├─ YYYY_MM_pairwise_metrics.json               (방법쌍 비교 지표)
      └─ YYYY_MM_full_result.json                    (모든 원시 결과/지표 종합)
"""

import os, re, json, math, calendar, joblib, warnings
from typing import List, Dict, Tuple
from datetime import datetime
from collections import Counter, defaultdict
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import spearmanr

# 외부 라이브러리 (설치 필요)
from krwordrank.word import KRWordRank
import yake
from keybert import KeyBERT

# =========================
# 경로 및 공통 설정
# =========================
file_path = '/home/ds4_sia_nolb/#FINAL_POLARIS/04_plus_preprocessing/preprocessing_final_data/final_preprocessing.json'
BENCH_OUTDIR = '/home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks'
os.makedirs(BENCH_OUTDIR, exist_ok=True)

TFIDF_VECTORIZER_PATH = '/home/ds4_sia_nolb/#FINAL_POLARIS/05_Event_top10/idf_vectorizer_for_all_corpus.pkl'
TOP_K = 30

warnings.filterwarnings("ignore")

# =========================
# 불용어/정규화
# =========================
BASE_STOP = {
    '가','간','같은','같이','것','게다가','결국','곧','관하여','관련','관한','그','그것','그녀','그들',
    '그리고','그때','그래','그래서','그러나','그러므로','그러한','그런','그렇게','그외','근거로','기타',
    '까지도','까지','나','남들','너','누구','다','다가','다른','다만','다소','다수','다시','다음','단','단지',
    '당신','대','대해서','더군다나','더구나','더라도','더욱이','도','도로','또','또는','또한','때','때문',
    '라도','라면','라는','로','로부터','로써','를','마저','마치','만약','만일','만큼','모두','무엇','무슨',
    '무척','물론','및','밖에','바로','보다','뿐이다','사람','사실은','상대적으로','생각','설령','소위','수',
    '수준','쉽게','시대','시작하여','실로','실제','아니','아무','아무도','아무리','아마도','아울러','아직',
    '앞에서','앞으로','어느','어떤','어떻게','어디','언제','얼마나','여기','여부','역시','예','오히려',
    '와','왜','외에도','요','우리','우선','원래','위해서','으로','으로부터','으로써','을','의','의거하여',
    '의지하여','의해','의해서','의하여','이','이것','이곳','이때','이라고','이러한','이런','이렇게','이제',
    '이지만','이후','이상','이다','이전','인','일','일단','일반적으로','임시로','입장에서','자','자기','자신',
    '잠시','저','저것','저기','저쪽','저희','전부','전혀','점에서','정도','제','조금','좀','주로','주제','즉',
    '즉시','지금','진짜로','차라리','참','참으로','첫번째로','최고','최대','최소','최신','최초','통하여',
    '통해서','평가','포함한','포함하여','하지만','하면서','하여','한','한때','한번','할','할것이다','할수있다',
    '함께','해도'
}
NEWS_STOP = {"기자","연합뉴스","사진","속보","종합","자료","영상","단독","전문","인터뷰","브리핑"}
CUSTOM_STOPWORDS = BASE_STOP | NEWS_STOP

ENTITY_NOISE = {
    "북한","한국","대한민국","남한","미국","중국","일본","러시아","우크라이나","유엔","나토","NATO","EU","유럽연합",
    "푸틴","블라디미르 푸틴","바이든","조 바이든","시진핑","김정은","김여정","문재인","윤석열","쇼이구","젤렌스키","중앙","통신","보도",
    '돼다','서다','대해','나오다','통해','맞다'
}

def normalize_text(t: str) -> str:
    if not t:
        return ""
    t = t.replace("탄도 미사일","탄도미사일").replace("순항 미사일","순항미사일")
    t = t.replace("극초 음속","극초음속").replace("초대형 방사포","초대형방사포")
    # 공백 정리
    t = re.sub(r"\s+"," ", t).strip()
    return t

# =========================
# 날짜/입력 로더
# =========================
def parse_date_flexible(s: str):
    if not s or not isinstance(s, str):
        return None
    s = s.strip()
    cands = [s]
    if "T" in s:
        cands += [s[:19], s[:10]]
    if len(s) >= 10:
        cands.append(s[:10])
    if "-" not in s and "." not in s and "/" not in s and len(s) == 8:
        cands.append(f"{s[:4]}-{s[4:6]}-{s[6:8]}")
    fmts = [
        "%Y-%m-%d","%Y-%m-%d %H:%M:%S","%Y-%m-%d %H:%M",
        "%Y/%m/%d","%Y/%m/%d %H:%M:%S",
        "%Y.%m.%d","%Y.%m.%d %H:%M:%S","%Y.%m.%d %H:%M",
        "%Y%m%d","%Y-%m-%dT%H:%M:%S"
    ]
    for c in cands:
        for f in fmts:
            try:
                return datetime.strptime(c, f)
            except:
                pass
    return None

def extract_pubdate(a: dict):
    keys = ["pubDate","pubdate","time","date","published","pub_date"]
    for k in keys:
        if k in a and a[k]:
            dt = parse_date_flexible(str(a[k]))
            if dt: return dt
    meta = a.get("metadata", {}) or {}
    for k in keys:
        if k in meta and meta[k]:
            dt = parse_date_flexible(str(meta[k]))
            if dt: return dt
    return None

def doc_text(a: dict) -> str:
    title = (a.get('metadata') or {}).get('title','')
    summary = a.get('summary','') or ''
    return normalize_text(f"{title} {summary}")

def load_all_articles(path: str) -> List[dict]:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError("JSON 루트는 list 여야 합니다.")
    return data

def monthly_corpus(year: int, month: int, all_articles: List[dict]) -> List[str]:
    last_day = calendar.monthrange(year, month)[1]
    sdt = datetime(year, month, 1)
    edt = datetime(year, month, last_day, 23, 59, 59)
    docs = []
    for a in all_articles:
        d = extract_pubdate(a)
        if d and sdt <= d <= edt:
            text = doc_text(a)
            if text:
                docs.append(text)
    return docs

# =========================
# TF-IDF (전체 코퍼스 학습/로드)
# =========================

# 기존 pkl 호환을 위한 별칭(aliased) - 이전 이름 유지
def tokenizer_for_vectorizer(s: str):
    return tokenizer_simple_ko(s)

def tokenizer_simple_ko(s: str) -> List[str]:
    # 아주 간단한 토크나이저: 한글/영문/숫자 단어 기준 + 길이>=2 + 불용어 제외
    toks = re.findall(r"[가-힣A-Za-z0-9]+", s)
    out = []
    for w in toks:
        if len(w) <= 1: continue
        if w in CUSTOM_STOPWORDS: continue
        if w.isdigit(): continue
        out.append(w)
    return out

def load_or_train_vectorizer(all_articles: List[dict], path: str) -> TfidfVectorizer:
    if os.path.exists(path):
        return joblib.load(path)
    corpus = [doc_text(a) for a in all_articles]
    vec = TfidfVectorizer(
        tokenizer=tokenizer_simple_ko,
        ngram_range=(1,3),
        min_df=5,
        max_df=0.85,
        sublinear_tf=True,
        norm='l2'
    )
    vec.fit(corpus)
    joblib.dump(vec, path)
    return vec

def tfidf_top_phrases(docs: List[str], vectorizer: TfidfVectorizer, top_k=30) -> List[Tuple[str, float]]:
    if not docs:
        return []
    X = vectorizer.transform(docs)
    avg = np.asarray(X.mean(axis=0)).ravel()
    terms = vectorizer.get_feature_names_out()
    pairs = [(terms[i], float(avg[i])) for i in np.where(avg>0)[0]]
    # 엔터티 노이즈 약벌
    def penalty(term):
        toks = term.split()
        ent_hits = sum(1 for t in toks if any(ent in t for ent in ENTITY_NOISE))
        return -0.05 * ent_hits
    scored = [(t, s + penalty(t)) for t,s in pairs]
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:top_k]

# =========================
# TextRank (KRWordRank) → 단어스코어로 구(phrase) 스코어
# =========================
from krwordrank.word import KRWordRank

def textrank_top_phrases(docs: List[str], top_k=30) -> List[Tuple[str, float]]:
    if not docs:
        return []
    # 문서 수가 적으면 min_count를 낮춰야 키워드가 나옵니다.
    min_count = 5 if len(docs) >= 100 else 2

    kr = KRWordRank(min_count=min_count, max_length=10, verbose=False)

    # beta(0~1): 텔레포테이션 가중, max_iter: 반복
    try:
        # 신버전 호환 (delta 지원)
        keywords, rank, _ = kr.extract(docs, beta=0.85, max_iter=50, delta=0.001)
    except TypeError:
        # 구버전 호환 (delta 미지원)
        keywords, rank, _ = kr.extract(docs, beta=0.85, max_iter=50)

    # 1~3그램 phrase 스코어링 (단어 rank 합산)
    phrases = Counter()
    for text in docs:
        words = re.findall(r"[가-힣A-Za-z0-9]+", text)
        words = [w for w in words if w not in CUSTOM_STOPWORDS and len(w) > 1]
        for n in (1, 2, 3):
            for i in range(len(words) - n + 1):
                ph = " ".join(words[i:i+n])
                # 엔터티 노이즈 과다 포함 구 제외
                if n <= 2 and any(ent in ph for ent in ENTITY_NOISE):
                    continue
                score = sum(rank.get(w, 0.0) for w in words[i:i+n])
                if score > 0:
                    phrases[ph] += score

    scored = list(phrases.items())
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:top_k]

# =========================
# YAKE
# =========================
def yake_top_phrases(docs: List[str], top_k=30) -> List[Tuple[str, float]]:
    if not docs:
        return []
    text = "\n".join(docs)
    # 낮은 점수가 더 좋음 → 1/score로 뒤집어 정렬
    kw_extractor = yake.KeywordExtractor(
        lan="ko", n=3, # 1~3그램 자동 탐색
        dedupLim=0.9, windowsSize=1, top=top_k*3,
        features=None
    )
    candidates = kw_extractor.extract_keywords(text)
    # 후보 정리: 불용어/숫자/짧은 토큰 제거
    cleaned = []
    for phrase, score in candidates:
        ph = " ".join([w for w in re.findall(r"[가-힣A-Za-z0-9]+", phrase) if len(w)>1 and w not in CUSTOM_STOPWORDS])
        if not ph: continue
        cleaned.append((ph, score))
    # 중복 축약 (동일 phrase는 최고 점수만 남김)
    best = {}
    for ph, sc in cleaned:
        inv = 1.0/max(sc, 1e-9)
        best[ph] = max(best.get(ph, 0.0), inv)
    ranked = sorted(best.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

# =========================
# KeyBERT (멀티링구얼 사전학습 임베딩)
# =========================
_KEYBERT_MODEL = None
def get_keybert():
    global _KEYBERT_MODEL
    if _KEYBERT_MODEL is None:
        # 다국어 모델 (가벼움, ko 지원)
        _KEYBERT_MODEL = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')
    return _KEYBERT_MODEL

def keybert_top_phrases(docs: List[str], top_k=30) -> List[Tuple[str, float]]:
    if not docs:
        return []
    text = "\n".join(docs)
    kw = get_keybert()
    # KeyBERT 점수: cos sim (높을수록 좋음)
    candidates = kw.extract_keywords(
        text,
        keyphrase_ngram_range=(1,3),
        stop_words=list(CUSTOM_STOPWORDS),
        top_n=top_k*5
    )
    # 정리 + 중복 제거
    agg = {}
    for ph, sc in candidates:
        ph2 = " ".join([w for w in re.findall(r"[가-힣A-Za-z0-9]+", ph) if len(w)>1 and w not in CUSTOM_STOPWORDS])
        if not ph2: continue
        agg[ph2] = max(agg.get(ph2, 0.0), float(sc))
    ranked = sorted(agg.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

# =========================
# 비교 지표/출력
# =========================
def to_rank_dict(items: List[Tuple[str, float]]) -> Dict[str, int]:
    return {ph: i for i,(ph,_) in enumerate(items, start=1)}

def jaccard(a: List[str], b: List[str]) -> float:
    sa, sb = set(a), set(b)
    if not sa and not sb: return 1.0
    if not sa or not sb: return 0.0
    return len(sa & sb) / len(sa | sb)

def token_jaccard(a: List[str], b: List[str]) -> float:
    ta = set(sum([ph.split() for ph in a], []))
    tb = set(sum([ph.split() for ph in b], []))
    if not ta and not tb: return 1.0
    if not ta or not tb: return 0.0
    return len(ta & tb) / len(ta | tb)

def spearman_on_common(a_items: List[Tuple[str,float]], b_items: List[Tuple[str,float]]) -> float:
    ra, rb = to_rank_dict(a_items), to_rank_dict(b_items)
    common = [ph for ph in ra if ph in rb]
    if len(common) < 3:
        return float('nan')
    xa = [ra[ph] for ph in common]
    xb = [rb[ph] for ph in common]
    rho, _ = spearmanr(xa, xb)
    return float(rho)

def intersec_top(a_items, b_items, top=15) -> List[Tuple[str, int, int]]:
    ra, rb = to_rank_dict(a_items), to_rank_dict(b_items)
    common = [(ph, ra[ph], rb[ph]) for ph in ra if ph in rb]
    common.sort(key=lambda x: (x[1]+x[2]))
    return common[:top]

def save_csv_per_method(year, month, results: Dict[str, List[Tuple[str,float]]], outdir=BENCH_OUTDIR):
    rows = []
    for m, items in results.items():
        for rank, (ph, sc) in enumerate(items, start=1):
            rows.append({"year":year, "month":month, "method":m, "rank":rank, "phrase":ph, "score":sc})
    path = os.path.join(outdir, f"{year}_{month:02d}_methods_keywords.csv")
    # CSV 직접 작성(표준 라이브러리)
    import csv
    with open(path, "w", encoding="utf-8", newline="") as f:
        wr = csv.DictWriter(f, fieldnames=["year","month","method","rank","phrase","score"])
        wr.writeheader()
        wr.writerows(rows)
    print(f"📄 저장: {path}")

def compare_methods(year:int, month:int, all_articles: List[dict], vectorizer: TfidfVectorizer):
    docs = monthly_corpus(year, month, all_articles)
    print(f"📚 {year}-{month:02d} 문서 수: {len(docs)}")
    if not docs:
        print("⚠️ 문서가 없습니다.")
        return

    results = {}
    # 1) TF-IDF
    results["tfidf"]    = tfidf_top_phrases(docs, vectorizer, TOP_K)
    # 2) TextRank
    results["textrank"] = textrank_top_phrases(docs, TOP_K)
    # 3) YAKE
    results["yake"]     = yake_top_phrases(docs, TOP_K)
    # 4) KeyBERT
    results["keybert"]  = keybert_top_phrases(docs, TOP_K)

    # 쌍별 비교 지표
    methods = list(results.keys())
    pairwise = {}
    for i in range(len(methods)):
        for j in range(i+1, len(methods)):
            a, b = methods[i], methods[j]
            a_items, b_items = results[a], results[b]
            a_ph = [p for p,_ in a_items]
            b_ph = [p for p,_ in b_items]
            pairwise[f"{a}_vs_{b}"] = {
                "jaccard_phrase": round(jaccard(a_ph, b_ph), 3),
                "jaccard_token": round(token_jaccard(a_ph, b_ph), 3),
                "spearman_rank_on_common": (None if math.isnan(spearman_on_common(a_items, b_items)) else round(spearman_on_common(a_items, b_items), 3)),
                "intersection_top": [
                    {"phrase": ph, "rank_in_"+a: ra, "rank_in_"+b: rb}
                    for ph, ra, rb in intersec_top(a_items, b_items, top=15)
                ]
            }

    # 저장
    save_csv_per_method(year, month, results, BENCH_OUTDIR)

    # JSON 종합 저장
    out_full = {
        "year": year, "month": month,
        "n_docs": len(docs),
        "top_k": TOP_K,
        "results": {
            m: [{"phrase": ph, "score": float(sc)} for ph, sc in items]
            for m, items in results.items()
        },
        "pairwise_metrics": pairwise
    }
    jpath = os.path.join(BENCH_OUTDIR, f"{year}_{month:02d}_full_result.json")
    with open(jpath, "w", encoding="utf-8") as f:
        json.dump(out_full, f, ensure_ascii=False, indent=2)
    print(f"💾 저장: {jpath}")

    # 별도: pairwise만 저장
    ppath = os.path.join(BENCH_OUTDIR, f"{year}_{month:02d}_pairwise_metrics.json")
    with open(ppath, "w", encoding="utf-8") as f:
        json.dump(out_full["pairwise_metrics"], f, ensure_ascii=False, indent=2)
    print(f"💾 저장: {ppath}")

def main():
    # 1) 전체 기사 로드
    all_articles = load_all_articles(file_path)
    print(f"전체 문서 수: {len(all_articles)}")

    # 2) TF-IDF 벡터라이저 로드/학습
    vectorizer = load_or_train_vectorizer(all_articles, TFIDF_VECTORIZER_PATH)
    print("TF-IDF 벡터라이저 준비 완료")

    # 3) 실행 대상 연월 설정 (예: 2024년 1~12월)
    year = 2024
    target_months = list(range(1,13))

    for m in target_months:
        print("\n" + "="*70)
        print(f"▶ {year}년 {m}월 비교 실행")
        compare_methods(year, m, all_articles, vectorizer)

if __name__ == "__main__":
    main()


전체 문서 수: 80810
TF-IDF 벡터라이저 준비 완료

▶ 2024년 1월 비교 실행
📚 2024-01 문서 수: 510


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📄 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_01_methods_keywords.csv
💾 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_01_full_result.json
💾 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_01_pairwise_metrics.json

▶ 2024년 2월 비교 실행
📚 2024-02 문서 수: 377
📄 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_02_methods_keywords.csv
💾 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_02_full_result.json
💾 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_02_pairwise_metrics.json

▶ 2024년 3월 비교 실행
📚 2024-03 문서 수: 350
📄 저장: /home/ds4_sia_nolb/#FINAL_POLARIS/08_performance_evaluation/issue_performance_data/benchmarks/2024_03_methods_keywords.csv
💾 저장: /home/ds4_sia_nolb/#F